In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

开始使用端点和共享VM

<table align="left">
  <td style="text-align: center">
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/get_started_with_vertex_endpoint_and_shared_vm.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
        </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Fcustom%2Fget_started_with_vertex_endpoint_and_shared_vm.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>   
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/get_started_with_vertex_endpoint_and_shared_vm.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/get_started_with_vertex_endpoint_and_shared_vm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub中查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在 Google Cloud 上使用 Vertex AI 进行端到端 MLOps 生产。本教程涵盖了如何开始使用 Endpoints 和共享 VM 以共同托管模型。

了解有关[跨部署共享资源](https://cloud.google.com/vertex-ai/docs/predictions/model-co-hosting)的更多信息。

### 目标

在本教程中，您将学习如何使用部署资源池来部署模型。部署资源池提供了在同一（共享）VM 上托管多个模型的能力。

部署资源池将模型部署组合到一起，以在VM内共享资源。多个端点可以部署在同一个VM中的一个部署资源池中。这些端点中的每一个可以拥有一个或多个已部署的模型。对于给定端点的已部署模型可以分组在相同或不同的部署资源池中。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Training`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 将预训练的图像分类模型上传为`Model`资源（模型A）。
- 将预训练的文本句子编码器模型上传为`Model`资源（模型B）。
- 创建一个共享的VM部署资源池。
- 列出共享的VM部署资源池。
- 创建两个`Endpoint`资源。
- 使用部署资源池将第一个模型（模型A）部署到第一个`Endpoint`资源。
- 使用部署资源池将第二个模型（模型B）部署到第二个`Endpoint`资源。
- 使用第一个已部署模型（模型A）进行预测请求。
- 使用第二个已部署模型（模型B）进行预测请求。

### 模型

本教程中使用的预训练模型来自[TensorFlow Hub](https://tfhub.dev/)存储库：

- [图像分类](https://tfhub.dev/google/imagenet/inception_v3/classification/5)：使用ImageNet进行训练。
- [文本句子编码器](https://tfhub.dev/google/universal-sentence-encoder/4)：谷歌的通用句子编码器。

### 费用
本教程使用谷歌云的付费组件：

- Vertex AI
- 云存储

请了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预计使用情况生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他所需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 tensorflow==2.15.1 \
                                 tensorflow-hub==0.16.1

重新启动运行时（仅限Colab）

要使用新安装的包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将要重新启动。在继续下一步之前，请等待重新启动完成。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置Google Cloud项目信息并初始化Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并启用Vertex AI API。了解更多关于设置项目和开发环境的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type: "string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在的情况下才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION $BUCKET_URI

### 导入库

In [ ]:
import os

import google.cloud.aiplatform as aiplatform
import google.cloud.aiplatform_v1beta1 as aip_beta
import tensorflow as tf
import tensorflow_hub as hub

### 初始化顶点AI Python SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Vertex AI 常量

为 Vertex AI 设置以下常量：

- `API_ENDPOINT`：用于 `Endpoint` 服务的 Vertex AI API 服务端点。
- `PARENT`：您项目和位置中所有 Vertex AI 资源的基本资源路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(LOCATION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + LOCATION

## 设置客户端

Vertex AI 以客户端/服务器模型工作。在您的一侧（Python 脚本），您创建一个客户端，用于向 Vertex AI 服务器发送请求并接收响应。

在这个教程中，您需要为工作流程中的不同步骤使用不同的客户端。这意味着您需要在开始时设置它们。

- Endpoint 服务用于创建端点，并将模型部署到端点。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_endpoint_client():
    client = aip_beta.EndpointServiceClient(client_options=client_options)
    return client


clients = {}
clients["endpoint"] = create_endpoint_client()

for client in clients.items():
    print(client)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器映像以及分配给虚拟机实例（VM）的 GPU 数。例如，为了使用一个支持 GPU 的容器映像，并将4个 Nvidia Telsa T4 GPU 分配给每个 VM，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，请指定 `(None, None)` 来使用一个可以在 CPU 上运行的容器映像。

了解更多关于 [您位置的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建容器

为预测设置预构建的Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.13".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：\ [2、4、8、16、32、64、96 \]的数量

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了来自TensorFlow Hub（TFHub）的预训练模型，然后将其上传到`Vertex AI Model`资源中。一旦您拥有了`Vertex AI Model`资源，该模型可以部署到`Vertex AI Endpoint`资源中。

### 下载预训练的图像分类模型

首先，您从TensorFlow Hub下载预训练的图像分类模型。该模型作为TF.Keras层进行下载。为了完成模型，在这个例子中，您创建了一个带有下载的TFHub模型作为层的`Sequential()`模型，并指定了模型的输入形状。下载的模型在ImageNet上进行了预训练。

In [ ]:
tfhub_model_icn = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5")]
)
tfhub_model_icn.build([None, 224, 224, 3])

在这一点上，模型在内存中。接下来，将模型工件保存到云存储位置。

In [ ]:
MODEL_ICN_DIR = BUCKET_URI + "/model_icn"
tfhub_model_icn.save(MODEL_ICN_DIR)

下一步，您需要将TFHub图像分类模型上传到Vertex AI“Model”服务，该服务为您的模型创建了一个Vertex AI“Model”资源。在上传过程中，您需要定义一个serving function来将数据转换为模型期望的格式。如果您将编码数据发送到Vertex AI，您的serving function将确保数据在传递到模型之前在模型服务器上解码。

### serving function如何工作

当您向在线预测服务器发送请求时，请求将被HTTP服务器接收。HTTP服务器从HTTP请求内容正文中提取出预测请求。提取出的预测请求将转发给serving function。对于Google预构建的预测容器，请求内容将作为`tf.string`传递给serving function。

serving function由两部分组成：

- `预处理函数`:
  - 将输入（`tf.string`）转换为基础模型（动态图）期望的输入形状和数据类型。
  - 执行与在训练基础模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`:
  - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，创建JSON对象等。

预处理和后处理函数都被转换为与模型融合的静态图。基础模型的输出传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在构建TF.Keras模型的serving函数时需要考虑的一点是，它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，则在服务函数的编译过程中会出现错误，指示您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

#### 预处理

为了将图像传递给预测服务，您需要将压缩（例如JPEG）图像字节编码为Base 64，以使内容在通过网络传输二进制数据时安全免受修改。由于此部署模型期望输入数据为原始（未压缩）字节，您需要确保Base 64编码的数据被转换回原始字节，然后经过预处理以匹配模型输入要求，然后再将其作为输入传递给部署的模型。

为了解决这个问题，您需要定义一个服务功能（`serving_fn`），并将其附加到模型作为预处理步骤。添加一个`@tf.function`修饰符，以便服务功能与底层模型融合（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容会被Base 64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务功能（`serving_fn`）。服务功能将`tf.string`预处理为原始（未压缩）的Numpy字节（`preprocess_fn`），以匹配模型的输入要求：

- `io.decode_jpeg`- 解压缩JPG图像，将其作为一个具有三个通道（RGB）的Tensorflow张量返回。
- `image.convert_image_dtype`- 将整数像素值更改为float32，并在0到1之间重新缩放像素数据。
- `image.resize`- 调整图像大小以匹配模型的输入形状。

此时，数据可以通过具体功能传递给模型（`m_call`）。服务功能是一个静态图，而模型是一个动态图。具体功能执行将输入数据从服务功能传送到模型，以及将模型的预测结果从模型传送回服务功能的任务。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model_icn.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    tfhub_model_icn, MODEL_ICN_DIR, signatures={"serving_default": serving_fn}
)

获取服务函数签名

您可以通过重新加载模型到内存中，并查询与每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要服务函数的签名。为什么呢？当您将数据发送为预测的 HTTP 请求数据包时，图像数据是 base64 编码的，而您的 TF.Keras 模型需要 numpy 输入。您的服务函数将会将 base64 转换为一个 numpy 数组。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要了解服务函数的输入层名称 -- 这在之后进行预测请求时会用到。

In [ ]:
loaded = tf.saved_model.load(MODEL_ICN_DIR)

serving_input_icn = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input_icn)

将TensorFlow Hub模型上传到`Vertex AI模型`资源

最后，您将TFHub模型和服务功能中的模型工件上传到`Vertex AI模型`资源中。

*注意:*当您将模型工件上传到`Vertex AI模型`资源时，您需要指定相应的部署容器映像。

In [ ]:
model_icn = aiplatform.Model.upload(
    display_name="icn",
    artifact_uri=MODEL_ICN_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model_icn)

### 下载预先训练好的句子编码器模型

接下来，您可以从TensorFlow Hub下载预先训练好的文本句子编码器模型。该模型将以TF.Keras层的形式下载。为了完成模型，在本例中，您可以创建一个包含下载的TFHub模型作为层的`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model_use = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")]
)

# force the model to build
tfhub_model_use.predict(["foo"])

### 保存模型工件

目前，模型已经在内存中。 接下来，您需要将模型工件保存到云存储位置。

In [ ]:
MODEL_USE_DIR = BUCKET_URI + "/model_use"
tfhub_model_use.save(MODEL_USE_DIR)

获取服务功能签名

您可以通过重新加载模型到内存中，并查询相应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要服务功能的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称 - 这在您进行预测请求时会用到。

In [ ]:
loaded = tf.saved_model.load(MODEL_USE_DIR)

serving_input_use = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input_use)

### 将TensorFlow Hub模型上传到 `Vertex AI Model` 资源

最后，您将TFHub模型的模型工件和serving功能上传到 `Vertex AI Model` 资源中。

*注意:* 当您将模型工件上传到 `Vertex AI Model` 资源时，您需要指定相应的部署容器镜像。

In [ ]:
model_use = aiplatform.Model.upload(
    display_name="use",
    artifact_uri=MODEL_USE_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model_use)

## 创建部署资源池

目前，仅通过基于REST的API（例如，CURL）支持创建部署资源池。

使用`CreateDeploymentResourcePool` API来创建一个资源池，具有以下配置：

- `dedicated_resources`：为共享虚拟机分配的计算（硬件）资源。
- `min_replica_count`：自动扩展，计算节点的最小数量。
- `max_replica_count`：自动扩展，计算节点的最大数量。

了解更多关于[部署资源池](https://cloud.google.com/vertex-ai/docs/predictions/model-co-hosting)。

In [ ]:
DEPLOYMENT_RESOURCE_POOL_ID = "shared-vm"  # @param {type: "string"}

In [ ]:
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

MIN_NODES = 1
MAX_NODES = 2

CREATE_RP_PAYLOAD = {
  "deployment_resource_pool":{
    "dedicated_resources":{
      "machine_spec":{
        "machine_type": DEPLOY_COMPUTE
      },
      "min_replica_count": MIN_NODES, 
      "max_replica_count": MAX_NODES
    }
  },
  "deployment_resource_pool_id":DEPLOYMENT_RESOURCE_POOL_ID
}
CREATE_RP_REQUEST=json.dumps(CREATE_RP_PAYLOAD)
pp.pprint("CREATE_RP_REQUEST: " + CREATE_RP_REQUEST)

! curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/deploymentResourcePools \
-d '{CREATE_RP_REQUEST}'

获取部署资源池

使用`GetDeploymentResourcePool` API来查看您创建的部署资源池。

了解更多关于[获取部署资源池](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.deploymentResourcePools/get)。

In [ ]:
! curl -X GET \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/deploymentResourcePools/{DEPLOYMENT_RESOURCE_POOL_ID}

列出所有部署资源池

使用`ListDeploymentResourcePools` API来列出所有的部署资源池。

了解更多关于[列出部署资源池](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.deploymentResourcePools/list)。

In [ ]:
! curl -X GET \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/deploymentResourcePools

## 创建两个 `Endpoint` 资源

接下来，您可以使用 `Endpoint.create()` 方法创建两个 `Endpoint` 资源。最少需要指定端点的显示名称。可选地，您可以指定项目和位置（location）；否则，设置将继承您在使用 `init()` 方法初始化 Vertex AI SDK for Python 时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint` 资源的可读名称。

此方法返回一个 `Endpoint` 对象。

了解更多关于 [Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint_icn = aiplatform.Endpoint.create(display_name="icn")

print(endpoint_icn)

endpoint_use = aiplatform.Endpoint.create(display_name="use")

print(endpoint_use)

## 在部署资源池中部署模型

在创建模型和端点之后，您可以使用DeployModel API进行部署。请查看下面的CURL命令示例。请注意，您需要使用创建的资源池的部署资源名称来指定DeployedModel的`shared_resources`。

可以同时启动相同部署资源池中的模型部署。

### 部署图像分类模型

接下来，您可以将图像分类模型部署到一个`Endpoint`上，并使用您的部署资源池。

In [ ]:
SHARED_RESOURCE = "projects/{project_id}/locations/{location}/deploymentResourcePools/{deployment_resource_pool_id}".format(
    project_id=PROJECT_ID,
    location=LOCATION,
    deployment_resource_pool_id=DEPLOYMENT_RESOURCE_POOL_ID,
)

DEPLOY_MODEL_PAYLOAD = {
    "deployedModel": {
        "model": model_icn.resource_name,
        "shared_resources": SHARED_RESOURCE,
        "enable_container_logging": True,
    },
    "trafficSplit": {"0": 100},
}
DEPLOY_MODEL_REQUEST = json.dumps(DEPLOY_MODEL_PAYLOAD)
pp.pprint("DEPLOY_MODEL_REQUEST: " + DEPLOY_MODEL_REQUEST)

In [ ]:
ENDPOINT_ID = endpoint_icn.name

output = ! curl -X POST \
 -H "Authorization: Bearer $(gcloud auth print-access-token)" \
 -H "Content-Type: application/json" \
https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:deployModel \
-d '{DEPLOY_MODEL_REQUEST}'

for line in output:
    if '"name"' in line:
        operation_id = line.split(":")[-1].strip()[:-1]
        print(operation_id)
        break

等待部署完成

接着，您查询操作的状态，并等待操作状态`done`被设置为`true`。

In [ ]:
import time

done = False
while done != '"done": true':
    status = ! curl -X GET \
 -H "Authorization: Bearer $(gcloud auth print-access-token)" \
 -H "Content-Type: application/json" \
https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{operation_id}
    for line in status:
        if '"done"' in line.strip():
            done = line.strip()[0:-1]
    print("DONE status:", done)
    time.sleep(30)

### 部署文本句子编码模型

接下来，您可以使用部署资源池将文本句子编码模型部署到一个 `Endpoint`。

In [ ]:
DEPLOY_MODEL_PAYLOAD = {
    "deployedModel": {
        "model": model_use.resource_name,
        "shared_resources": SHARED_RESOURCE,
        "enable_container_logging": True,
    },
    "trafficSplit": {"0": 100},
}
DEPLOY_MODEL_REQUEST = json.dumps(DEPLOY_MODEL_PAYLOAD)
pp.pprint("DEPLOY_MODEL_REQUEST: " + DEPLOY_MODEL_REQUEST)

In [ ]:
ENDPOINT_ID = endpoint_use.name

output = ! curl -X POST \
 -H "Authorization: Bearer $(gcloud auth print-access-token)" \
 -H "Content-Type: application/json" \
https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:deployModel \
-d '{DEPLOY_MODEL_REQUEST}'

for line in output:
    if '"name"' in line:
        operation_id = line.split(":")[-1].strip()[:-1]
        print(operation_id)
        break

等待部署完成

接下来，您查询操作的状态，等待操作状态 `done` 被设置为 `true`。

In [ ]:
done = False
while done != '"done": true':
    status = ! curl -X GET \
 -H "Authorization: Bearer $(gcloud auth print-access-token)" \
 -H "Content-Type: application/json" \
https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{operation_id}
    for line in status:
        if '"done"' in line.strip():
            done = line.strip()[0:-1]
    print("DONE status:", done)
    time.sleep(30)

### 创建测试示例图像分类模型

接下来，测试您部署的图像分类模型。首先，将您的测试数据编码为用于服务功能的格式：

`{ serving_input: { 'b64': base64编码的字节 } }`

In [ ]:
! gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg test.jpg

import base64

with open("test.jpg", "rb") as f:
    data = f.read()
b64str = base64.b64encode(data).decode("utf-8")

最后，您发出一个预测请求。由于模型是在ImageNet上训练的，预测会返回对应的1000个类别的概率。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input_icn: {"b64": b64str}}]

prediction = endpoint_icn.predict(instances=instances)

print(prediction)

创建一个测试示例文本句子编码器模型

接下来，您需要测试已部署的文本句子编码器模型。首先，您需要为提供功能对测试数据进行编码，数据格式如下：

`"word1 word2 ... wordN"`

In [ ]:
instance = "the brown fox jumped over the laxy dog"

### 为文本句子编码器模型进行预测请求

最后，您发出一个预测请求。预测会返回一个包含500个元素的嵌入向量。

In [ ]:
endpoint_use.predict([instance])

清理资源

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

# Undeploy the models
# When you're done doing predictions, you undeploy the model from the `Endpoint` resouce. 
# This deprovisions all compute resources and ends billing for the deployed model.
endpoint_icn.undeploy_all()
endpoint_use.undeploy_all()

# Delete the `Endpoint` resources
# The method 'delete()' deletes the model.
model_icn.delete()
model_use.delete()

# Delete the `Endpoint` resources
# The method 'delete()' deletes the endpoint.
endpoint_icn.delete()
endpoint_use.delete()

# Delete the `DeploymentResourcePool`
# The method 'delete()' deletes your deployment resource pool.
! curl -X DELETE \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/deploymentResourcePools/{DEPLOYMENT_RESOURCE_POOL_ID}

!rm -f test.jpg